## 🎯 Practice Exercises
## Exercise 1: Build Your First Stateful Agent

**Difficulty:** Beginner
**Estimated Time:** 30-45 minutes

### Task
Build a simple customer support chatbot that remembers conversation context.

### Requirements
1. Create a StateGraph with MessagesState
2. Add a system prompt that makes the agent act as a helpful customer support rep
3. Use MemorySaver checkpointer for memory
4. Test with a multi-turn conversation where context matters

### Example Conversation
```
User: "I bought a laptop last week"
Agent: "I'd be happy to help with your laptop! What seems to be the issue?"
User: "It won't turn on"
Agent: "I understand your laptop won't turn on. Have you tried..."
```


In [1]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
import os

print("✅ All imports successful")

c:\Users\dell\Desktop\publica_jan\jenv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


✅ All imports successful


In [2]:
# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print("✅ API key loaded successfully")

✅ API key loaded successfully


In [3]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [5]:
# System prompt that defines assistant behavior
sys_msg = SystemMessage(
    content="You are a simple customer support chatbot that remembers conversation context."
)

def assistant(state: MessagesState) -> dict:
    """
    The assistant node - processes messages and generates response.
    """
    # Combine system prompt with conversation history
    messages = [sys_msg] + state["messages"]
    
    # Get response from LLM
    response = llm.invoke(messages)
    
    # Return as state update
    return {"messages": [AIMessage(content=response.content)]}

print("✅ Assistant node defined")

✅ Assistant node defined


In [6]:
# Create a StateGraph with MessagesState
builder = StateGraph(MessagesState)

# Add the assistant node
builder.add_node("assistant", assistant)

# Define the flow:
# START → assistant → END
builder.add_edge(START, "assistant")
builder.add_edge("assistant", END)

print("✅ Graph structure defined")

✅ Graph structure defined


In [7]:
# Create a memory checkpointer (stores in memory)
memory = MemorySaver()

# Compile the graph WITH memory
agent = builder.compile(checkpointer=memory)

print("✅ Agent compiled with memory")

✅ Agent compiled with memory


In [8]:
# Define a session ID for this conversation
session_id = "chat-session-0012"

print(f"Starting conversation with session ID: {session_id}")

Starting conversation with session ID: chat-session-0012


In [9]:
def run_conversation(user_input: str, thread_id: str = session_id):
    """
    Send a message to the agent and get response.
    ⚠️ WARNING: Using default thread_id shares conversation acrosss all calls!
    In production, ALWAYS provide unique thread_id per user.
    """
    # Invoke the agent
    result = agent.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"thread_id": thread_id}}
    )
    
    # Print the conversation
    for message in result["messages"]:
        if isinstance(message, HumanMessage):
            print(f"\n👤 User: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"🤖 Agent: {message.content}")
    
    print("\n" + "="*70)

print("✅ Conversation function ready")

✅ Conversation function ready


In [10]:
run_conversation("I bought a laptop last week")


👤 User: I bought a laptop last week
🤖 Agent: That's great! How is your new laptop working for you? Do you have any questions or need assistance with it?



In [11]:
run_conversation("It won't turn on")


👤 User: I bought a laptop last week
🤖 Agent: That's great! How is your new laptop working for you? Do you have any questions or need assistance with it?

👤 User: It won't turn on
🤖 Agent: I'm sorry to hear that! Here are a few steps you can try to troubleshoot the issue:

1. **Check the Power Supply**: Make sure the laptop is plugged in and the power adapter is connected properly. Look for any indicator lights on the laptop.

2. **Remove the Battery (if applicable)**: If your laptop has a removable battery, try taking it out and then plugging in the laptop without the battery. 

3. **Perform a Hard Reset**: If it’s a removable battery, unplug the power adapter, remove the battery, and press and hold the power button for about 15-20 seconds. Reinsert the battery and try to turn it on again.

4. **Check the Display**: Sometimes the laptop might be on but the screen is off. Try connecting it to an external monitor to see if there's a display issue.

If none of these steps work, it might 